Task from Netcracker to find outliners in data. Made by Alex Bocharov skype bam271074



Let s install pyldavis lib for visualization

In [1]:
!pip install pyldavis

     |████████████████████████████████| 1.7MB 5.8MB/s 
  Created wheel for pyldavis: filename=pyLDAvis-3.2.2-py2.py3-none-any.whl size=135593 sha256=e23b61d6d301459fe6cd957dd749f81ce30536a284b2c60ef5dbb37d4dfdcc90
  Stored in directory: /root/.cache/pip/wheels/74/df/b6/97234c8446a43be05c9a8687ee0db1f1b5ade5f27729187eae
Successfully built pyldavis


In [2]:
import numpy as np
import pandas as pd
import re
from sklearn.metrics import *
from sklearn.pipeline import *
from sklearn.feature_extraction.text import *
from sklearn.decomposition import TruncatedSVD, PCA
from sklearn.preprocessing import Normalizer, LabelEncoder
from gensim.models import *
from gensim import corpora
from gensim import similarities
import random
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import style
# %matplotlib inline
style.use('ggplot')
import pyLDAvis
import pyLDAvis.gensim as gensimvis


import warnings
warnings.filterwarnings('ignore')

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


In [3]:
#let s upload file with data
from google.colab import files
uploaded = files.upload()
for fn in uploaded.keys():
  print ('User uploaded file {name} with length {length} bytes'.format(name=fn, \
                                                                       length=len(uploaded[fn])))

Saving census-income-full-nominal.arff to census-income-full-nominal.arff
User uploaded file census-income-full-nominal.arff with length 136760796 bytes


In [4]:
!ls

census-income-full-nominal.arff  sample_data


In [5]:
!wget https://github.com/GuansongPang/anomaly-detection-datasets/blob/main/categorical%20data/census-income-full-nominal.tar.xz

--2021-03-02 12:01:29--  https://github.com/GuansongPang/anomaly-detection-datasets/blob/main/categorical%20data/census-income-full-nominal.tar.xz
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘census-income-full-nominal.tar.xz’

census-income-full-     [ <=>                ]  97.11K  --.-KB/s    in 0.04s   

2021-03-02 12:01:29 (2.42 MB/s) - ‘census-income-full-nominal.tar.xz’ saved [99441]



In [6]:
!ls

census-income-full-nominal.arff  census-income-full-nominal.tar.xz  sample_data


In [7]:
#let s build dataframe from uploaded file

df = pd.read_csv("census-income-full-nominal.arff", sep=',', skip_blank_lines=True, header=36)
df.head()

,Not-in-universe,0,0.1,Bachelors-degree(BA-AB-BS),Not-in-universe.1,Married-civilian-spouse-present,Not-in-universe-or-children,Not-in-universe.2,White,All-other,Female,Not-in-universe.3,Not-in-universe.4,Children-or-Armed-Forces,Joint-both-under-65,Not-in-universe.5,Not-in-universe.6,Spouse-of-householder,Spouse-of-householder.1,Nonmover,Nonmover.1,Nonmover.2,Yes,Not-in-universe.7,Not-in-universe.8,United-States,United-States.1,United-States.2,Native--Born-in-the-United-States,0.2,Not-in-universe.9,2,94,--50000.
0,Private,29,32,Bachelors-degree(BA-AB-BS),Not-in-universe,Never-married,Transportation,Other-service,White,All-other,Female,Not-in-universe,Job-leaver,Unemployed-full-time,Single,Not-in-universe,Not-in-universe,Nonfamily-householder,Householder,?,?,?,Not-in-universe-under-1-year-old,?,Not-in-universe,United-States,United-States,United-States,Native--Born-in-the-United-States,0,Not-in-universe,2,95,--50000.
1,Not-in-universe,0,0,High-school-graduate,Not-in-universe,Married-civilian-spouse-present,Not-in-universe-or-children,Not-in-universe,White,All-other,Male,Not-in-universe,Not-in-universe,Not-in-labor-force,Joint-both-65+,Not-in-universe,Not-in-universe,Householder,Householder,?,?,?,Not-in-universe-under-1-year-old,?,Not-in-universe,United-States,United-States,United-States,Native--Born-in-the-United-States,0,Not-in-universe,2,95,--50000.
2,Not-in-universe,0,0,Children,Not-in-universe,Never-married,Not-in-universe-or-children,Not-in-universe,White,All-other,Female,Not-in-universe,Not-in-universe,Children-or-Armed-Forces,Nonfiler,Not-in-universe,Not-in-universe,Grandchild-<18-never-marr-child-of-subfamily-RP,Other-relative-of-householder,?,?,?,Not-in-universe-under-1-year-old,?,Both-parents-present,United-States,United-States,United-States,Native--Born-in-the-United-States,0,Not-in-universe,0,95,--50000.
3,Not-in-universe,0,0,Children,Not-in-universe,Never-married,Not-in-universe-or-children,Not-in-universe,Asian-or-Pacific-Islander,All-other,Male,Not-in-universe,Not-in-universe,Children-or-Armed-Forces,Nonfiler,Not-in-universe,Not-in-universe,Child-<18-never-marr-not-in-subfamily,Child-under-18-never-married,Nonmover,Nonmover,Nonmover,Yes,Not-in-universe,Both-parents-present,China,China,United-States,Native--Born-in-the-United-States,0,Not-in-universe,0,94,--50000.
4,Not-in-universe,0,0,11th-grade,Not-in-universe,Widowed,Not-in-universe-or-children,Not-in-universe,White,All-other,Female,Not-in-universe,Not-in-universe,Children-or-Armed-Forces,Nonfiler,South,District-of-Columbia,Other-Rel-18+-ever-marr-RP-of-subfamily,Other-relative-of-householder,MSA-to-MSA,Different-region,Different-state-in-South,No,No,Not-in-universe,?,?,?,Foreign-born--Not-a-citizen-of-U-S,0,Not-in-universe,2,94,--50000.


In [8]:
df.tail()

,Not-in-universe,0,0.1,Bachelors-degree(BA-AB-BS),Not-in-universe.1,Married-civilian-spouse-present,Not-in-universe-or-children,Not-in-universe.2,White,All-other,Female,Not-in-universe.3,Not-in-universe.4,Children-or-Armed-Forces,Joint-both-under-65,Not-in-universe.5,Not-in-universe.6,Spouse-of-householder,Spouse-of-householder.1,Nonmover,Nonmover.1,Nonmover.2,Yes,Not-in-universe.7,Not-in-universe.8,United-States,United-States.1,United-States.2,Native--Born-in-the-United-States,0.2,Not-in-universe.9,2,94,--50000.
299279,Private,36,27,High-school-graduate,Not-in-universe,Divorced,Private-household-services,Private-household-services,White,All-other,Female,Not-in-universe,Not-in-universe,Full-time-schedules,Single,Not-in-universe,Not-in-universe,Householder,Householder,?,?,?,Not-in-universe-under-1-year-old,?,Not-in-universe,United-States,United-States,United-States,Native--Born-in-the-United-States,0,Not-in-universe,2,95,--50000.
299280,Private,35,17,Some-college-but-no-degree,Not-in-universe,Divorced,Finance-insurance-and-real-estate,Sales,White,All-other,Female,Not-in-universe,Job-loser---on-layoff,Unemployed-full-time,Head-of-household,Not-in-universe,Not-in-universe,Householder,Householder,?,?,?,Not-in-universe-under-1-year-old,?,Not-in-universe,United-States,United-States,United-States,Native--Born-in-the-United-States,0,Not-in-universe,2,95,--50000.
299281,Not-in-universe,0,0,Children,Not-in-universe,Never-married,Not-in-universe-or-children,Not-in-universe,White,All-other,Male,Not-in-universe,Not-in-universe,Children-or-Armed-Forces,Nonfiler,Not-in-universe,Not-in-universe,Child-<18-never-marr-not-in-subfamily,Child-under-18-never-married,Nonmover,Nonmover,Nonmover,Yes,Not-in-universe,Both-parents-present,United-States,United-States,United-States,Native--Born-in-the-United-States,0,Not-in-universe,0,94,--50000.
299282,Not-in-universe,0,0,Children,Not-in-universe,Never-married,Not-in-universe-or-children,Not-in-universe,Asian-or-Pacific-Islander,All-other,Female,Not-in-universe,Not-in-universe,Children-or-Armed-Forces,Nonfiler,Not-in-universe,Not-in-universe,Child-<18-never-marr-not-in-subfamily,Child-under-18-never-married,?,?,?,Not-in-universe-under-1-year-old,?,Both-parents-present,Hong-Kong,Hong-Kong,United-States,Native--Born-in-the-United-States,0,Not-in-universe,0,95,--50000.
299283,Not-in-universe,0,0,Children,Not-in-universe,Never-married,Not-in-universe-or-children,Not-in-universe,White,All-other,Female,Not-in-universe,Not-in-universe,Children-or-Armed-Forces,Nonfiler,Not-in-universe,Not-in-universe,Child-<18-never-marr-not-in-subfamily,Child-under-18-never-married,?,?,?,Not-in-universe-under-1-year-old,?,Both-parents-present,United-States,United-States,United-States,Native--Born-in-the-United-States,0,Not-in-universe,0,95,--50000.


In [9]:
df.shape

(299284, 34)

In [10]:
df.columns

Index(['Not-in-universe', '0', '0.1', 'Bachelors-degree(BA-AB-BS)',
       'Not-in-universe.1', 'Married-civilian-spouse-present',
       'Not-in-universe-or-children', 'Not-in-universe.2', 'White',
       'All-other', 'Female', 'Not-in-universe.3', 'Not-in-universe.4',
       'Children-or-Armed-Forces', 'Joint-both-under-65', 'Not-in-universe.5',
       'Not-in-universe.6', 'Spouse-of-householder', 'Spouse-of-householder.1',
       'Nonmover', 'Nonmover.1', 'Nonmover.2', 'Yes', 'Not-in-universe.7',
       'Not-in-universe.8', 'United-States', 'United-States.1',
       'United-States.2', 'Native--Born-in-the-United-States', '0.2',
       'Not-in-universe.9', '2', '94', '--50000.'],
      dtype='object')

In [11]:
df.describe(include=['object']).T

,count,unique,top,freq
Not-in-universe,299284,9,Not-in-universe,150322
Bachelors-degree(BA-AB-BS),299284,17,High-school-graduate,72553
Not-in-universe.1,299284,3,Not-in-universe,280366
Married-civilian-spouse-present,299284,7,Never-married,129628
Not-in-universe-or-children,299284,24,Not-in-universe-or-children,150965
Not-in-universe.2,299284,15,Not-in-universe,150965
White,299284,5,White,250977
All-other,298005,9,All-other,257829
Female,299284,2,Female,155773
Not-in-universe.3,299284,3,Not-in-universe,270741


In [12]:
#let s see how many NaNs
df.isnull().sum()

Not-in-universe                         0
0                                       0
0.1                                     0
Bachelors-degree(BA-AB-BS)              0
Not-in-universe.1                       0
Married-civilian-spouse-present         0
Not-in-universe-or-children             0
Not-in-universe.2                       0
White                                   0
All-other                            1279
Female                                  0
Not-in-universe.3                       0
Not-in-universe.4                       0
Children-or-Armed-Forces                0
Joint-both-under-65                     0
Not-in-universe.5                       0
Not-in-universe.6                       0
Spouse-of-householder                   0
Spouse-of-householder.1                 0
Nonmover                                0
Nonmover.1                              0
Nonmover.2                              0
Yes                                     0
Not-in-universe.7                 

Anoimaly #1- 1279 NaNs

In [13]:
df['All-other'].fillna(value='NaNvalue',inplace=True) 

In [ ]:
#let s see how many NaNs
df.isnull().sum()

Not-in-universe                      0
0                                    0
0.1                                  0
Bachelors-degree(BA-AB-BS)           0
Not-in-universe.1                    0
Married-civilian-spouse-present      0
Not-in-universe-or-children          0
Not-in-universe.2                    0
White                                0
All-other                            0
Female                               0
Not-in-universe.3                    0
Not-in-universe.4                    0
Children-or-Armed-Forces             0
Joint-both-under-65                  0
Not-in-universe.5                    0
Not-in-universe.6                    0
Spouse-of-householder                0
Spouse-of-householder.1              0
Nonmover                             0
Nonmover.1                           0
Nonmover.2                           0
Yes                                  0
Not-in-universe.7                    0
Not-in-universe.8                    0
United-States            

In [14]:
def concat_features(x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28,x29,x30,x31,x32,x33):
  """
  Let s concat features
  """
  buff=(str(x1)+' '+str(x2)+' '+str(x3)+' '+' '+str(x4)+' '+str(x5)+' '+str(x6)+' '+str(x7)+' '+str(x8)+' '+str(x9)+' '+
        str(x10)+' '+str(x11)+' '+str(x12)+' '+str(x13)+' '+str(x14)+' '+str(x15)+' '+str(x16)+' '+str(x17)+' '+
        str(x18)+' '+str(x19)+' '+str(x20)+' '+str(x21)+' '+str(x22)+' '+str(x23)+' '+str(x24)+' '+str(x25)+' '+
        str(x26)+' '+str(x27)+' '+str(x28)+' '+str(x29)+' '+str(x30)+' '+str(x31)+' '+str(x32)+' '+str(x33))
  return buff

In [15]:
concat_features('a','b','c','D','e','f,','r','y','f','h','ee',
                'w','e','d','r','t','w','x','i','u','o','l',
                'a','d','s','x','z','x','v','b','b','m','p')

'a b c  D e f, r y f h ee w e d r t w x i u o l a d s x z x v b b m p'

In [16]:
TOKEN_RE = re.compile(r'[\w\d]+')  #regular expression to divide data

In [17]:
def tokenize_text_simple_regex(txt, min_token_size=2):
    """ This func tokenize text with TOKEN_RE applied ealier """
    txt = txt.lower()
    all_tokens = TOKEN_RE.findall(txt)
    all_tokens = [token for token in all_tokens if len(token) >= min_token_size]
    s=' '.join(all_tokens)
    return s

In [18]:
X_list=[]  #list for concat results
#loop on dataframe
#we do not use column #34. it seems target
for t in df.itertuples():
  buff=concat_features(t[1],t[2],t[3],t[4],t[5],t[6],t[7],t[8],t[9],t[10],t[11],t[12],t[13],t[14],t[15],t[16],t[17],
                       t[18],t[19],t[20],t[21],t[22],t[23],t[24],t[25],t[26],t[27],t[28],t[29],t[30],t[31],t[32],t[33])
  buff=tokenize_text_simple_regex(buff, min_token_size=4)
  X_list.append(buff)

In [19]:
X_list[0]

'private bachelors degree universe never married transportation other service white other female universe leaver unemployed full time single universe universe nonfamily householder householder universe under year universe united states united states united states native born united states universe'

In [20]:
tokenized_texts = []
for text in X_list:
    #text = [w for w in text.split() if w not in sw]  #del stop words
    text = [w for w in text.split()]
    tokenized_texts.append(text)

In [21]:
print('Making dictionary...')
dictionary = corpora.Dictionary(tokenized_texts)
print('Original: {}'.format(dictionary))
dictionary.filter_extremes(no_below = 2, no_above = 0.95, keep_n=None)
print('Filtered: {}'.format(dictionary))

print('Vectorizing corpus...')
corpus = [dictionary.doc2bow(text) for text in tokenized_texts]

Making dictionary...
Original: Dictionary(285 unique tokens: ['bachelors', 'born', 'degree', 'female', 'full']...)
Filtered: Dictionary(283 unique tokens: ['bachelors', 'degree', 'female', 'full', 'householder']...)
Vectorizing corpus...


##Latent Dirichet Allocation(LDA)


This model is also implemented in the gensim library. One of its advantages is that you can train the finished model (and unlike LSA and pLSA, where even when adding one new document you have to train the model from scratch).

We assume that in the entire collection of k topics.
We distribute these k topics according to documents m (this is the distribution α), assigning a topic to each document.
For each word w in the document, we assume that its theme is incorrect, while the themes of all other words are correct.

Based on probability, we assign a theme to the word w, based on:
which topics in document m
how many times the word w was assigned to specific topics in the entire collection (this is the distribution β)
We repeat many, many times
α is a matrix (stochastic) where each row = document, column = theme.

β is a matrix (stochastic) where each row = theme, column = word.

## Latent Dirichet Allocatoin

Данная модель также реализована в библиотеке `gensim`. Один из ее плюсов в том, что можно дообучать готовую модель (а отличие от LSA и pLSA, где даже при добавлении одного нового документа приходится обучать модель с нуля).

- Предполагаем, что во всей коллекции k тем.
- Распределяем эти k тем по документам m (это распределение α), присваивая каждому документу тему.
- Для каждого слова w  в документе mпредполагаем, что его тема неверна, при этом темы всех остальных слов верны.
- Основываясь на вероятности присваиваем слову w тему, опираясь на:
    1. какие темы в документе  m
    2. сколько раз слово w присваивалось конкретным темам во всей коллекции (это распределение β)
- Повторяем много-много раз

α - это матрица (стохастическая), где каждая строка = документ, столбец = тема. 

β - это матрица (стохастическая), где каждая строка = тема, столбец = слово.

In [24]:
%%time
lda = ldamodel.LdaModel(corpus, id2word=dictionary, num_topics=25, chunksize=50, update_every=1, passes=2)

CPU times: user 3min 59s, sys: 242 ms, total: 4min
Wall time: 4min


In [25]:
lda.show_topics(num_topics=3, num_words=15, formatted=False)

[(3,
  [('repair', 0.21305153),
   ('precision', 0.1412185),
   ('production', 0.1412185),
   ('craft', 0.1412185),
   ('construction', 0.07788097),
   ('male', 0.06742022),
   ('private', 0.06360922),
   ('indian', 0.025378028),
   ('aleut', 0.025378028),
   ('amer', 0.025378028),
   ('eskimo', 0.025378028),
   ('married', 0.017473033),
   ('householder', 0.015768496),
   ('grade', 0.011482829),
   ('mining', 0.008056941)]),
 (16,
  [('handlers', 0.18143915),
   ('cleaners', 0.18143915),
   ('equip', 0.18143915),
   ('trade', 0.17443115),
   ('wholesale', 0.13492459),
   ('canada', 0.12878646),
   ('scotland', 0.017241675),
   ('carolina', 1.0822305e-06),
   ('west', 1.0822305e-06),
   ('virginia', 1.0822305e-06),
   ('poland', 1.0822305e-06),
   ('12th', 1.0822305e-06),
   ('diploma', 1.0822305e-06),
   ('guatemala', 1.0822305e-06),
   ('entrant', 1.0822305e-06)]),
 (10,
  [('some', 0.11096987),
   ('college', 0.10412853),
   ('degree', 0.10373141),
   ('householder', 0.0745544),
   

In [26]:
vis_data = gensimvis.prepare(lda, corpus, dictionary)
pyLDAvis.display(vis_data)

For example if we take a look at theme #28, it is connected with people from Cuba(take a look at words connected with this group). We found anomaly.